In [ ]:
%load_ext autoreload
%autoreload 2

# Run conversion script for Raw_Intan_Ephys_withVideos

In [ ]:
from nelson_lab_to_nwb.intan_sessions_converter import session_to_nwb


session_to_nwb(
    output_folder_path="output_intan",
    intan_folder_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205_240205_120954",
    aim_score_file_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/AIM_score.xlsx",
    top_behavioral_video_file_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205Top.mp4",
    side_behavioral_video_file_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205Side.mp4",
    user_metadata_file_path="/mnt/shared_storage/Github/nelson-lab-to-nwb/src/nelson_lab_to_nwb/intan_sessions_converter/metadata_example.yaml",
    injection_time_in_seconds=0.0,
    stub_test=True,
    overwrite=True,
    verbose=True,
)

In [ ]:
from pynwb import NWBHDF5IO, NWBFile
from pathlib import Path


file_path = str(Path("ouput_intan/subid012345_20240205T120954.nwb").resolve())
io = NWBHDF5IO(file_path, 'r', load_namespaces=True)
nwbfile = io.read()
nwbfile

In [ ]:
from nelson_lab_to_nwb.intan_sessions_converter import IntanSessionNWBConverter
from neuroconv.utils.dict import load_dict_from_file, dict_deep_update
from pynwb import NWBHDF5IO, NWBFile

In [ ]:
# set the source data for the Converter
source_data = dict(
    IntanMultifilesRaw=dict(
        folder_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205_240205_120954"
    ),
    AIMScore=dict(
        file_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/AIM_score.xlsx",
    ),
    BehavioralVideoTop=dict(
        file_paths=["/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205Top.mp4"],
    ),
    BehavioralVideoSide=dict(
        file_paths=["/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205Side.mp4"],
    ),
)

# Instantiate the Converter
converter = IntanSessionNWBConverter(source_data)

# Automatically fetch metadata from files, then update it with user-defined metadata
source_metadata = converter.get_metadata()
user_metadata_file = "/mnt/shared_storage/Github/nelson-lab-to-nwb/src/nelson_lab_to_nwb/intan_sessions_converter/metadata_example.yaml"
user_metadata = load_dict_from_file(file_path=user_metadata_file)
metadata = dict_deep_update(source_metadata, user_metadata)

In [ ]:
metadata

In [ ]:
# Conversion options
conversion_options = dict(
    IntanMultifilesRaw=dict(
        stub_test=True,
    )
    AIMScore=dict(
        timestamp_column_name="Time (minutes relative to injection)",
        aims_column_name="AIMS",
    )
)

# Run conversion
converter.run_conversion(
    metadata=metadata,
    nwbfile_path="converted_intan_session.nwb",
    overwrite=True,
    conversion_options=conversion_options
)

In [ ]:
io = NWBHDF5IO("converted_intan_session.nwb", 'r', load_namespaces=True)
nwbfile = io.read()
nwbfile

# Multiple Intan files

ref: https://github.com/SpikeInterface/spikeinterface/issues/2620

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nelson_lab_to_nwb.interfaces.intan_multi_interface import IntanMultifilesRecordingInterface
from datetime import datetime
from dateutil import tz
from pynwb import NWBHDF5IO, NWBFile
import numpy as np

In [ ]:
folder_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205_240205_120954"
interface = IntanMultifilesRecordingInterface(folder_path=folder_path, verbose=False)

# Extract what metadata we can from the source files
metadata = interface.get_metadata()
# session_start_time is required for conversion. If it cannot be inferred
# automatically from the source files you must supply one.
session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=tz.gettz("US/Pacific"))
metadata["NWBFile"].update(session_start_time=session_start_time)

In [ ]:
nwbfile_path = "intan_data.nwb"
interface.run_conversion(
    nwbfile_path=nwbfile_path,
    metadata=metadata,
    overwrite=True
)

In [ ]:
io = NWBHDF5IO("intan_data.nwb", 'r', load_namespaces=True)
nwbfile = io.read()
nwbfile

In [ ]:
ext = interface.recording_extractor
ext

In [ ]:
ext.

In [ ]:
from neuroconv.datainterfaces import IntanRecordingInterface

file_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205_240205_120954/pm231213a_240205_240205_120954.rhd"
file_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205_240205_120954/pm231213a_240205_240205_141354.rhd"
interface_0 = IntanRecordingInterface(file_path=file_path, verbose=False)

interface_0.get_metadata()

In [ ]:
ext_0 = interface_0.recording_extractor
ext_0

In [ ]:
ne = ext_0.neo_reader
ne

In [ ]:
s0 = ne.get_analogsignal_chunk(
    block_index=0,
    seg_index=0,
    stream_index=0
)
s0.shape

In [ ]:
s1 = ne.get_analogsignal_chunk(
    block_index=0,
    seg_index=0,
    stream_index=1
)
s1.shape

In [ ]:
s1

In [ ]:
s2 = ne.get_analogsignal_chunk(
    block_index=0,
    seg_index=0,
    stream_index=2
)
s2.shape

In [ ]:
np.unique(np.squeeze(s2))

In [ ]:
s3 = ne.get_analogsignal_chunk(
    block_index=0,
    seg_index=0,
    stream_index=3
)
s3.shape

In [ ]:
import cv2

def get_frame_count(video_path):
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print("Error: Could not open video.")
        return None
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    video.release()
    return frame_count


video_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205Top.mp4"
frame_count = get_frame_count(video_path)
print(f"Number of frames in the video: {frame_count}")

# AIM Score

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nelson_lab_to_nwb.interfaces import AIMScoreInterface

In [ ]:
file_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/AIM_score.xlsx"
interface = AIMScoreInterface(
    file_path=file_path,
    # timestamp_column_name="Time (minutes relative to injection)",
    # aims_column_name="AIMS",
    verbose=True
)

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

metadata = interface.get_metadata()
session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=ZoneInfo("US/Pacific"))
metadata["NWBFile"].update(session_start_time=session_start_time)

In [ ]:
metadata

In [ ]:
interface.create_nwbfile(
    metadata=metadata,
    timestamp_offset=1000.
)

# Multi videos

In [ ]:
from neuroconv import NWBConverter
from datetime import datetime
from zoneinfo import ZoneInfo
from neuroconv.datainterfaces import VideoInterface


class MultiVideoConverter(NWBConverter):
    data_interface_classes = dict(
        BehavioralVideoTop=VideoInterface,
        BehavioralVideoSide=VideoInterface,
    )

source_data = dict(
    BehavioralVideoTop=dict(
        file_paths=["/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205Top.mp4"],
        metadata_key_name="VideoTop"
    ),
    BehavioralVideoSide=dict(
        file_paths=["/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205Side.mp4"],
        metadata_key_name="VideoSide"
    ),
)

converter = MultiVideoConverter(source_data)

metadata = converter.get_metadata()
session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=ZoneInfo("US/Pacific"))
metadata["NWBFile"].update(session_start_time=session_start_time)

print(metadata["Behavior"])
print(converter.get_metadata_schema()["properties"]["Behavior"]["properties"].keys())

converter.run_conversion(
    metadata=metadata,
    nwbfile_path="converted_multi_videos_session.nwb",
    overwrite=True
)

In [ ]:
io = NWBHDF5IO("converted_multi_videos_session.nwb", 'r', load_namespaces=True)
nwbfile = io.read()
nwbfile